### 3.3 클래스 정의

이미 정의된 클래스에 더해 사용자 정의 클래스를 추가할 수 있다.

#### 트리 예

In [32]:
data BinTree a = Leaf a
               | BinTree a :^: BinTree a

In [33]:
data LabTree l a = Tip a
                 | LFork l (LabTree l a) (LabTree l a)

In [34]:
data STree a = Empty
             | Split a (STree a) (STree a)
             deriving (Eq, Show)

In [35]:
data RoseTree a = Node a [RoseTree a]

In [36]:
type Name = String
data Term = Var Name      -- variable
          | Ap  Term Term -- application
          | Lam Name Term -- lambda abstraction

서로 다른 형태의 트리지만 동일한 속성을 가질 수 있다. 

In [37]:
class Tree t where
  subtrees :: t -> [t]

각 트리 타입에 대해 instance를 만들어보면...

In [38]:
instance Tree (BinTree a) where
  subtrees (Leaf n) = []
  subtrees (l :^: r) = [l,r]

instance Tree (LabTree l a) where
  subtrees (Tip _) = []
  subtrees (LFork _ l r) = [l,r]

instance Tree (STree a) where
  subtrees Empty = []
  subtrees (Split _ l r) = [l,r]

instance Tree (RoseTree a) where
  subtrees (Node x gts) = gts

instance Tree Term where
  subtrees (Var _) = []
  subtrees (Ap f x) = [f,x]
  subtrees (Lam v b) = [b]

이제 트리 일반적인 오퍼레이션을 정의할 수 있다.

In [39]:
depth:: Tree t => t -> Int
depth = (1+) . foldl max 0 . map depth . subtrees

size:: Tree t => t -> Int
size = (1+) . sum . map size . subtrees

`BinTree`의 경우에는 `size`를 최적화 할 수 있다. (`subtrees`의 중간 결과 리스트를 만들 필요 없으므로..) 이런 specialization은 어떻게 가능할까? 

```haskell
size (Leaf _) = 1
size (l :^: r) = size l + size r
```

(*Dictionary-free overloading by partial evaluation*에서 가능한 방법이 설명되나 보다. 게다가 이 구현은 `subtrees`를 이용하는 경우와 다른 답을 내놓는다.)

In [40]:
t = Leaf 1 :^: Leaf 2

In [41]:
size t

3

In [42]:
bsize (Leaf _) =1
bsize (l :^: r) = bsize l + bsize r

In [43]:
bsize t

2

`size`, `depth` 외에 `paths`, `dfs`, `bfs` 등의 알고리즘도 공통적으로 구현할 수 있다.

In [44]:
paths:: Tree t => t -> [[t]]
paths t | null br = [[t]]
        | otherwise = [t:p | b<-br, p<-paths b]
         where br = subtrees t

dfs :: Tree t => t -> [t]
dfs t = t : concat (map dfs (subtrees t))

bfs :: Tree t => t -> [t]
bfs = concat . lev
 where lev t = [t] : foldr cat [] (map lev (subtrees t))
       cat = combine (++)

combine :: (a -> a -> a) -> ([a] -> [a] -> [a])
combine f (x:xs) (y:ys) = f x y : combine f xs ys
combine _ [] ys = ys
combine _ xs [] = xs

(`bfs`는 level-order 문제와 같으며, `foldr cat []` 대신 `map concat . transpose`를 이용할 수도 있겠다)

In [45]:
:m +Data.List
bfs' = concat . lev
 where lev t = [t] :  map concat (transpose (map lev (subtrees t)))

In [50]:
s1 = Split 2 (Split 1 Empty Empty) (Split 3 Empty (Split 4 Empty Empty))

In [51]:
bfs' s1 == bfs s1

True